Import packages

In [51]:
from utils import *
import numpy as np
import torch
from valda.valuation import DataValuation

Initialize the parameters

In [52]:
train_size = 1000
train_start = 200
train_size_batch = 200
dev_size = 100
test_size = 300
np.random.seed(0)
torch.manual_seed(0)

## Load Dataset
Load the MNIST dataset and split it into training and test sets.

In [53]:
X_train, y_train_clean, X_dev, y_dev, X_test, y_test = load_data(train_size, dev_size, test_size)
y_train, flip_indices = flip_labels(y_train_clean, flip_fraction=0.3)

Test the models on the poisoned data set

In [54]:
cnn_accuracy_poisoned, cnn_accuracies_poisoned = cnn_tests(X_train, y_train, X_dev, y_dev)
print('CNN accuracy on poisoned dataset: {}'.format(cnn_accuracy_poisoned))

logistic_accuracy_poisoned = logistic_test(X_train, y_train, X_dev, y_dev)
print('Logistic Regression accuracy on poisoned dataset: {}'.format(logistic_accuracy_poisoned))

Accuracy of CNN model on cuda: 0.76
Accuracy of CNN model on cuda: 0.69
Accuracy of CNN model on cuda: 0.67
Accuracy of CNN model on cuda: 0.71
Accuracy of CNN model on cuda: 0.77
Accuracy of CNN model on cuda: 0.73
Accuracy of CNN model on cuda: 0.66
Accuracy of CNN model on cuda: 0.68
CNN accuracy on poisoned dataset: 0.70875
Accuracy of Logistic Regression model: 0.38
Logistic Regression accuracy on poisoned dataset: 0.38


Test the models on the clean data set

In [55]:
cnn_accuracy_pure, cnn_accuracies_pure = cnn_tests(X_train, y_train_clean, X_dev, y_dev)
print('CNN accuracy on clean dataset: {}'.format(cnn_accuracy_pure))

logistic_accuracy_pure = logistic_test(X_train, y_train_clean, X_dev, y_dev)
print('Logistic Regression accuracy on clean dataset: {}'.format(logistic_accuracy_pure))

Accuracy of CNN model on cuda: 0.90
Accuracy of CNN model on cuda: 0.88
Accuracy of CNN model on cuda: 0.90
Accuracy of CNN model on cuda: 0.84
Accuracy of CNN model on cuda: 0.88
Accuracy of CNN model on cuda: 0.88
Accuracy of CNN model on cuda: 0.87
Accuracy of CNN model on cuda: 0.86
CNN accuracy on clean dataset: 0.87625
Accuracy of Logistic Regression model: 0.82
Logistic Regression accuracy on clean dataset: 0.82


## Running
Divide the train set into batches and run the valuation.

In [56]:
X_train_batch = X_train[train_start:train_start + train_size_batch]
y_train_batch = y_train[train_start:train_start + train_size_batch]

In [ ]:
# Define a DataValuation instance
dv = DataValuation(X_train_batch, y_train_batch, X_dev, y_dev)

# Run with the default setting:
# 1. Logistic regression as the classifier
# 2. Prediction accuracy as the value measurement
vals_tmc = dv.estimate(method="tmc-shapley")

 25%|██▍       | 124/500 [08:58<14:55,  2.38s/it] 

## Results
Get the indices of the data points with values < 0

In [ ]:
min_indices_batch = [key for key, value in vals_tmc.items() if value <= 0]
min_indices_batch = np.array(min_indices_batch) + train_start

Append the indices to a file

In [ ]:
if not os.path.exists('./data'):
    os.makedirs('./data')

file_path = f'./data/min_indices_{str(train_start).zfill(3)}_{str(train_start + train_size_batch - 1).zfill(3)}.npy'
np.save(file_path, min_indices_batch)